In [1]:
# ATSIT 2019: dimensional speech emotion recognition from text feature

# uncomment these to run on CPU only
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]=""

import numpy as np
import matplotlib.pyplot as plt
import pickle
import keras.backend as K
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, LSTM, TimeDistributed, Bidirectional, Embedding, Dropout, Flatten, concatenate, Convolution1D
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

from keras.callbacks import EarlyStopping
import codecs

import io
import random as rn
import tensorflow as tf

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss

#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')

# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# load fastext model
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

word_index = tokenizer.word_index
nb_words = len(word_index) +1
print('Found %s unique tokens' % len(word_index))

# load model
EMBEDDING_DIM = 300
file_loc = '../../dimensionalSM/data/glove.42B.300d.txt'
print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

# split train/test

earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)



Found 3437 unique tokens
../../dimensionalSM/data/glove.42B.300d.txt
G Word embeddings: 1917494
G Null word embeddings: 128


In [4]:
split = 7869

In [5]:
# model: GRU
def text_model1():
    inputs = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(inputs)
    #net = Bidirectional(LSTM(64, return_sequences=True))(net)
#     net = Bidirectional(LSTM(64, return_sequences=True))(net)
#     net = Bidirectional(LSTM(64, return_sequences=True))(net)
    net = Bidirectional(LSTM(128, return_sequences=True))(net)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    out_text = Dense(64)(net)
    #out_text = Dropout(0.5)(net)
    target_names = ('v', 'a', 'd')
    outputs = [Dense(1, name=name)(out_text) for name in target_names]

    model = Model(inputs=inputs, outputs=outputs) #=[out1, out2, out3])
    model.compile(loss=ccc_loss, #{'v': ccc_loss, 'a': ccc_loss, 'd': ccc_loss},
                     loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model
# model = text_model1()
# model.summary()

In [6]:
# #speech_to_npy
# model1 = text_model1()
# earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
#                               restore_best_weights=True)
# hist1 = model1.fit(x_train_text[:split], vad[:split].T.tolist(), epochs=50, 
#                    batch_size=32, verbose=1, validation_split=0.2, 
#                    callbacks=[earlystop])

# eval_metrik1 = model1.evaluate(x_train_text[split:], vad[split:].T.tolist())
# print(eval_metrik1)

# # make prediction
# predict = model1.predict(x_train_text[6290:], batch_size=32)
# np.save('text_npy7869', 
#          np.array(predict).reshape(3, 3749).T)


In [7]:
# main function for for LOSO
def main(time):
    model = text_model1()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model.fit( x_train_text[:split], 
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model.evaluate( x_train_text[split:], vad[split:].T.tolist())
    print(metrik[-3:])
    # make prediction
    predict = model.predict(x_train_text[6290:], batch_size=32)
    np.save('npy7869/text_npy-7869glove'+str(time)+'.npy',  
             np.array(predict).reshape(3, 3749).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    weight = []
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('npy7869/text_blstm7869_no_dropout-1.csv', header=['time','v','a','d', 'ave'])


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 47s 8ms/step - loss: 1.8808 - v_loss: 0.6011 - a_loss: 0.6397 - d_loss: 0.6397 - v_ccc: 0.3986 - a_ccc: 0.3602 - d_ccc: 0.3603 - val_loss: 1.9215 - val_v_loss: 0.6549 - val_a_loss: 0.6448 - val_d_loss: 0.6461 - val_v_ccc: 0.3516 - val_a_ccc: 0.3639 - val_d_ccc: 0.3630
Epoch 2/50
6295/6295 [==============================] - 45s 7ms/step - loss: 1.4538 - v_loss: 0.4081 - a_loss: 0.5262 - d_loss: 0.5196 - v_ccc: 0.5918 - a_ccc: 0.4737 - d_ccc: 0.4806 - val_loss: 1.8671 - val_v_loss: 0.6241 - val_a_loss: 0.6408 - val_d_loss: 0.6264 - val_v_ccc: 0.3815 - val_a_ccc: 0.3678 - val_d_ccc: 0.3836
Epoch 3/50
6295/6295 [==============================] - 44s 7ms/step - loss: 1.2729 - v_loss: 0.3358 - a_loss: 0.4696 - d_loss: 0.4673 - v_ccc: 0.6640 - a_ccc: 0.5304 - d_ccc: 0.5327 - val_loss: 1.8678 - val_v_loss: 0.59

Epoch 5/50
6295/6295 [==============================] - 43s 7ms/step - loss: 1.0386 - v_loss: 0.2571 - a_loss: 0.3932 - d_loss: 0.3884 - v_ccc: 0.7430 - a_ccc: 0.6068 - d_ccc: 0.6116 - val_loss: 1.8728 - val_v_loss: 0.5640 - val_a_loss: 0.6299 - val_d_loss: 0.7000 - val_v_ccc: 0.4403 - val_a_ccc: 0.3785 - val_d_ccc: 0.3084
Epoch 6/50
6295/6295 [==============================] - 44s 7ms/step - loss: 0.9395 - v_loss: 0.2297 - a_loss: 0.3589 - d_loss: 0.3519 - v_ccc: 0.7707 - a_ccc: 0.6416 - d_ccc: 0.6482 - val_loss: 1.9356 - val_v_loss: 0.5867 - val_a_loss: 0.6491 - val_d_loss: 0.7211 - val_v_ccc: 0.4202 - val_a_ccc: 0.3580 - val_d_ccc: 0.2862
Epoch 7/50
6295/6295 [==============================] - 43s 7ms/step - loss: 0.8440 - v_loss: 0.2044 - a_loss: 0.3245 - d_loss: 0.3150 - v_ccc: 0.7956 - a_ccc: 0.6753 - d_ccc: 0.6851 - val_loss: 1.8661 - val_v_loss: 0.5685 - val_a_loss: 0.6353 - val_d_loss: 0.6915 - val_v_ccc: 0.4410 - val_a_ccc: 0.3732 - val_d_ccc: 0.3197
Epoch 8/50
6295/6295 [===

6295/6295 [==============================] - 44s 7ms/step - loss: 0.4402 - v_loss: 0.1072 - a_loss: 0.1696 - d_loss: 0.1634 - v_ccc: 0.8928 - a_ccc: 0.8305 - d_ccc: 0.8365 - val_loss: 1.9398 - val_v_loss: 0.5721 - val_a_loss: 0.6875 - val_d_loss: 0.7058 - val_v_ccc: 0.4370 - val_a_ccc: 0.3177 - val_d_ccc: 0.3055
Epoch 16/50
6295/6295 [==============================] - 44s 7ms/step - loss: 0.4154 - v_loss: 0.0987 - a_loss: 0.1614 - d_loss: 0.1559 - v_ccc: 0.9015 - a_ccc: 0.8387 - d_ccc: 0.8444 - val_loss: 1.8855 - val_v_loss: 0.5622 - val_a_loss: 0.6738 - val_d_loss: 0.6779 - val_v_ccc: 0.4484 - val_a_ccc: 0.3315 - val_d_ccc: 0.3346
Epoch 17/50
2170/2170 [==============================] - 4s 2ms/step
[0.3909347355365753, 0.38333404064178467, 0.38012731075286865]
2
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 46s 7ms/step - loss: 1.8759 - v_loss: 0.5921 - a_loss: 0.6428 - d_loss: 0.6414 - v_ccc: 0.4080 - a_ccc: 0.3573 - d_ccc: 0.

6295/6295 [==============================] - 43s 7ms/step - loss: 1.4528 - v_loss: 0.4111 - a_loss: 0.5176 - d_loss: 0.5248 - v_ccc: 0.5891 - a_ccc: 0.4825 - d_ccc: 0.4755 - val_loss: 1.8641 - val_v_loss: 0.6403 - val_a_loss: 0.6199 - val_d_loss: 0.6296 - val_v_ccc: 0.3658 - val_a_ccc: 0.3900 - val_d_ccc: 0.3800
Epoch 3/50
6295/6295 [==============================] - 43s 7ms/step - loss: 1.2759 - v_loss: 0.3443 - a_loss: 0.4681 - d_loss: 0.4633 - v_ccc: 0.6556 - a_ccc: 0.5319 - d_ccc: 0.5367 - val_loss: 1.8049 - val_v_loss: 0.5726 - val_a_loss: 0.6154 - val_d_loss: 0.6414 - val_v_ccc: 0.4323 - val_a_ccc: 0.3943 - val_d_ccc: 0.3684
Epoch 4/50
6295/6295 [==============================] - 43s 7ms/step - loss: 1.1293 - v_loss: 0.2922 - a_loss: 0.4188 - d_loss: 0.4185 - v_ccc: 0.7080 - a_ccc: 0.5812 - d_ccc: 0.5815 - val_loss: 1.8719 - val_v_loss: 0.6099 - val_a_loss: 0.6309 - val_d_loss: 0.6592 - val_v_ccc: 0.3970 - val_a_ccc: 0.3800 - val_d_ccc: 0.3511
Epoch 5/50
6295/6295 [==============

Epoch 14/50
6295/6295 [==============================] - 45s 7ms/step - loss: 0.4791 - v_loss: 0.1163 - a_loss: 0.1819 - d_loss: 0.1810 - v_ccc: 0.8836 - a_ccc: 0.8182 - d_ccc: 0.8190 - val_loss: 1.8684 - val_v_loss: 0.5587 - val_a_loss: 0.6617 - val_d_loss: 0.6736 - val_v_ccc: 0.4503 - val_a_ccc: 0.3428 - val_d_ccc: 0.3385
Epoch 15/50
6295/6295 [==============================] - 45s 7ms/step - loss: 0.4440 - v_loss: 0.1101 - a_loss: 0.1722 - d_loss: 0.1618 - v_ccc: 0.8899 - a_ccc: 0.8279 - d_ccc: 0.8382 - val_loss: 1.8893 - val_v_loss: 0.5571 - val_a_loss: 0.6662 - val_d_loss: 0.6896 - val_v_ccc: 0.4519 - val_a_ccc: 0.3390 - val_d_ccc: 0.3199
Epoch 16/50
6295/6295 [==============================] - 45s 7ms/step - loss: 0.4214 - v_loss: 0.1041 - a_loss: 0.1624 - d_loss: 0.1547 - v_ccc: 0.8959 - a_ccc: 0.8376 - d_ccc: 0.8451 - val_loss: 1.8729 - val_v_loss: 0.5662 - val_a_loss: 0.6639 - val_d_loss: 0.6685 - val_v_ccc: 0.4436 - val_a_ccc: 0.3419 - val_d_ccc: 0.3416
Epoch 17/50
6295/6295 

...

Epoch 12/50
6295/6295 [==============================] - 40s 6ms/step - loss: 0.5486 - v_loss: 0.1349 - a_loss: 0.2076 - d_loss: 0.2057 - v_ccc: 0.8650 - a_ccc: 0.7922 - d_ccc: 0.7942 - val_loss: 1.8933 - val_v_loss: 0.5689 - val_a_loss: 0.6842 - val_d_loss: 0.6627 - val_v_ccc: 0.4356 - val_a_ccc: 0.3216 - val_d_ccc: 0.3495
Epoch 13/50
6295/6295 [==============================] - 40s 6ms/step - loss: 0.5085 - v_loss: 0.1257 - a_loss: 0.1939 - d_loss: 0.1894 - v_ccc: 0.8744 - a_ccc: 0.8065 - d_ccc: 0.8106 - val_loss: 1.8773 - val_v_loss: 0.5830 - val_a_loss: 0.6385 - val_d_loss: 0.6823 - val_v_ccc: 0.4259 - val_a_ccc: 0.3686 - val_d_ccc: 0.3282
Epoch 14/50
6295/6295 [==============================] - 40s 6ms/step - loss: 0.4726 - v_loss: 0.1169 - a_loss: 0.1817 - d_loss: 0.1740 - v_ccc: 0.8831 - a_ccc: 0.8183 - d_ccc: 0.8260 - val_loss: 1.8491 - val_v_loss: 0.5589 - val_a_loss: 0.6558 - val_d_loss: 0.6567 - val_v_ccc: 0.4459 - val_a_ccc: 0.3521 - val_d_ccc: 0.3529
Epoch 15/50
2170/2170 

6295/6295 [==============================] - 41s 7ms/step - loss: 0.8390 - v_loss: 0.2046 - a_loss: 0.3173 - d_loss: 0.3172 - v_ccc: 0.7953 - a_ccc: 0.6828 - d_ccc: 0.6830 - val_loss: 1.8972 - val_v_loss: 0.5565 - val_a_loss: 0.6562 - val_d_loss: 0.7016 - val_v_ccc: 0.4443 - val_a_ccc: 0.3508 - val_d_ccc: 0.3077
Epoch 8/50
6295/6295 [==============================] - 41s 6ms/step - loss: 0.7671 - v_loss: 0.1879 - a_loss: 0.2890 - d_loss: 0.2906 - v_ccc: 0.8126 - a_ccc: 0.7110 - d_ccc: 0.7093 - val_loss: 1.8074 - val_v_loss: 0.5450 - val_a_loss: 0.6404 - val_d_loss: 0.6422 - val_v_ccc: 0.4575 - val_a_ccc: 0.3676 - val_d_ccc: 0.3674
Epoch 9/50
6295/6295 [==============================] - 41s 6ms/step - loss: 0.7006 - v_loss: 0.1681 - a_loss: 0.2654 - d_loss: 0.2676 - v_ccc: 0.8321 - a_ccc: 0.7347 - d_ccc: 0.7325 - val_loss: 1.8314 - val_v_loss: 0.5634 - val_a_loss: 0.6445 - val_d_loss: 0.6542 - val_v_ccc: 0.4463 - val_a_ccc: 0.3634 - val_d_ccc: 0.3589
Epoch 10/50
6295/6295 [=============

6295/6295 [==============================] - 41s 7ms/step - loss: 0.5128 - v_loss: 0.1239 - a_loss: 0.1983 - d_loss: 0.1908 - v_ccc: 0.8762 - a_ccc: 0.8018 - d_ccc: 0.8093 - val_loss: 1.8799 - val_v_loss: 0.5494 - val_a_loss: 0.6647 - val_d_loss: 0.6942 - val_v_ccc: 0.4610 - val_a_ccc: 0.3427 - val_d_ccc: 0.3164
Epoch 14/50
6295/6295 [==============================] - 41s 6ms/step - loss: 0.4834 - v_loss: 0.1160 - a_loss: 0.1872 - d_loss: 0.1802 - v_ccc: 0.8841 - a_ccc: 0.8127 - d_ccc: 0.8198 - val_loss: 1.8683 - val_v_loss: 0.5600 - val_a_loss: 0.6541 - val_d_loss: 0.6846 - val_v_ccc: 0.4492 - val_a_ccc: 0.3539 - val_d_ccc: 0.3286
Epoch 15/50
2170/2170 [==============================] - 4s 2ms/step
[0.3911497890949249, 0.40270787477493286, 0.40809687972068787]
15
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 47s 7ms/step - loss: 1.8815 - v_loss: 0.6021 - a_loss: 0.6402 - d_loss: 0.6392 - v_ccc: 0.3977 - a_ccc: 0.3596 - d_ccc: 0

6295/6295 [==============================] - 41s 6ms/step - loss: 1.1289 - v_loss: 0.2886 - a_loss: 0.4261 - d_loss: 0.4140 - v_ccc: 0.7113 - a_ccc: 0.5739 - d_ccc: 0.5859 - val_loss: 1.8698 - val_v_loss: 0.6093 - val_a_loss: 0.6361 - val_d_loss: 0.6445 - val_v_ccc: 0.3930 - val_a_ccc: 0.3719 - val_d_ccc: 0.3653
Epoch 5/50
6295/6295 [==============================] - 40s 6ms/step - loss: 1.0285 - v_loss: 0.2556 - a_loss: 0.3899 - d_loss: 0.3834 - v_ccc: 0.7445 - a_ccc: 0.6104 - d_ccc: 0.6166 - val_loss: 1.8176 - val_v_loss: 0.5608 - val_a_loss: 0.6263 - val_d_loss: 0.6528 - val_v_ccc: 0.4412 - val_a_ccc: 0.3829 - val_d_ccc: 0.3583
Epoch 6/50
6295/6295 [==============================] - 40s 6ms/step - loss: 0.9426 - v_loss: 0.2362 - a_loss: 0.3561 - d_loss: 0.3506 - v_ccc: 0.7639 - a_ccc: 0.6441 - d_ccc: 0.6494 - val_loss: 1.8382 - val_v_loss: 0.5599 - val_a_loss: 0.6290 - val_d_loss: 0.6710 - val_v_ccc: 0.4436 - val_a_ccc: 0.3795 - val_d_ccc: 0.3387
Epoch 7/50
6295/6295 [==============

Epoch 14/50
6295/6295 [==============================] - 41s 7ms/step - loss: 0.4777 - v_loss: 0.1148 - a_loss: 0.1829 - d_loss: 0.1802 - v_ccc: 0.8852 - a_ccc: 0.8172 - d_ccc: 0.8199 - val_loss: 1.8812 - val_v_loss: 0.5646 - val_a_loss: 0.6557 - val_d_loss: 0.6841 - val_v_ccc: 0.4424 - val_a_ccc: 0.3512 - val_d_ccc: 0.3252
Epoch 15/50
6295/6295 [==============================] - 41s 7ms/step - loss: 0.4462 - v_loss: 0.1080 - a_loss: 0.1722 - d_loss: 0.1658 - v_ccc: 0.8920 - a_ccc: 0.8278 - d_ccc: 0.8340 - val_loss: 1.9030 - val_v_loss: 0.5644 - val_a_loss: 0.6681 - val_d_loss: 0.6962 - val_v_ccc: 0.4450 - val_a_ccc: 0.3382 - val_d_ccc: 0.3138
Epoch 16/50
6295/6295 [==============================] - 41s 7ms/step - loss: 0.4215 - v_loss: 0.1023 - a_loss: 0.1617 - d_loss: 0.1574 - v_ccc: 0.8977 - a_ccc: 0.8382 - d_ccc: 0.8426 - val_loss: 1.9765 - val_v_loss: 0.5940 - val_a_loss: 0.7140 - val_d_loss: 0.6906 - val_v_ccc: 0.4143 - val_a_ccc: 0.2915 - val_d_ccc: 0.3177
Epoch 17/50
2170/2170 